### Hashing 

A hash function is a function that takes an inpute and deterministically converts it to an integer that is less than a fized size set by the programmer. Inputs are called **keys** and the same input will always be converted to the same integer.

**Check if the sentence is Pangram**
- A pangram is a sentence where every letter of the English alphabet appears at least once.

In [2]:
#Q: Given a string sentence containing only lowercase English letters, return true if sentence is a pangram or false otherwise
def is_pangram(sentence):
    seen = set()

    for i in sentence:
        seen.add(i)
        if len(seen) == 26:
            return True 
        
    return len(seen) == 26 

if __name__ == '__main__':
    sentence = "thequickbrownfoxjumpsoverthelazydog"
    print(is_pangram(sentence))

True


### Create A HashTable Or Hash Function/ Hash Map

Python implementation of a simple hash table (also known as a hash map) using **separate chaning** to handle **collision** including basic operators or methods such as **set**, **get** and **delete** 

In [ ]:
class HashTable:
    def __init__(self, size=100):
        """ 
        Initialize the table with a fixed size.
        Each slot in the table contains a list to handle collision via chaining.
        """
        self.size = size 
        self.table = [[] for _ in range(self.size)]

    def _hash(self, key):
        """
        Generate a hash for the given key.
        Using the python built-in hash function 
        """
        return hash(key) % self.size